In [31]:
%pip install nltk
%pip install tweepy
%pip install pandas
%pip install wordcloud
%pip install textblob
%pip install seaborn
%pip install poyraz

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated pa

ERROR: Could not find a version that satisfies the requirement poyraz (from versions: none)
ERROR: No matching distribution found for poyraz


In [32]:
import tweepy
import json
import csv
import sys
import time
import re
import warnings
import string
import pandas as pd
import ast
import nltk
import poyraz
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import Stream
from wordcloud import WordCloud
from textblob import TextBlob
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'poyraz'

In [ ]:
class getTweets:
    def __init__(self,ckey,csecret,atoken,asecret):
        self.api=None
        self.consumer_key = ckey
        self.consumer_secret = csecret
        self.access_token = atoken
        self.access_token_secret = asecret
        
    def setAuth(self):
        try:
            self.auth = tweepy.OAuthHandler(self.consumer_key, self.consumer_secret)
            self.auth.set_access_token(self.access_token, self.access_token_secret)
            self.api = tweepy.API(self.auth)            
            print ("Bağlantı Yapıldı!")                           
        except tweepy.TweepError as err:
            print('Error: {}'.format(err))   
    
    def toDataFrame(tweets):
        try:
            print ("Veriler Kayıt Ediliyor...")
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', None)
            pd.set_option('display.max_colwidth', None)
            DataSet = pd.DataFrame()

            DataSet['Tweet_Id'] = [tweet.id for tweet in tweets]
            DataSet['Tweet'] = [tweet.text for tweet in tweets]
            DataSet['Tweet_Degistirilmis'] = [tweet.text for tweet in tweets]
            DataSet['Tweet_Ceviri']=[tweet.text for tweet in tweets]
            DataSet['Tweet_Tarih'] = [tweet.created_at for tweet in tweets]
            DataSet['Kullanıcı_Id'] = [tweet.user.id for tweet in tweets]  
            DataSet['Kullanıcı_Adı'] = [tweet.user.name for tweet in tweets]       
            DataSet['Dil'] = [tweet.lang for tweet in tweets]
            print ("Veriler Kayıt Edildi!")
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
    
    def searchTweet(self,aranan):
        self.setAuth()
        try:                      
            print ("Kelimeye Uygun Tweetler Sorgulanıyor...")
            cursor = tweepy.Cursor(self.api.search_tweets, q=aranan, result_type="new", geocode="55.0000,4.0000,1000km",lang='tr')
            results=[]
            for item in cursor.items(100):
                    results.append(item)
            print ("Kelimeye Uygun Tweetler Sorgulandı!")
            DataSet = getTweets.toDataFrame(results)  
            getTweets.TweetTemizle(DataSet)
            DataSet.to_csv('tw.csv',index=False)
        except Exception as err:
            print('Error: {}'.format(err))
                  
    def Ceviri(DataSet):
        try:
            print ("Tweetler Çeviriliyor...")
            for yazi in range(len(DataSet["Tweet"])):
                Ceviri_Blob=TextBlob(DataSet["Tweet"][yazi])
                Eng_Blob=Ceviri_Blob.translate(from_lang='tr', to='en')  
                DataSet['Tweet_Ceviri'][yazi]=Eng_Blob
            print ("Tweetler Çevirildi!")
            return DataSet        
        except Exception as err:
            print('Error: {}'.format(err))
            
    def TweetTemizle(Dataset):
        try:
            print ("Tweetler Temizleniyor...")
            pattern = re.compile('http[s]?://(?:[A-Za-z0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+|#[A-Za-z0-9]+|$[A-Za-z0-9]+|@[A-Za-z0-9_]+|[,.^_$*%-;鶯!?’"“”:=|…]+|\n')
            for i in range(len(Dataset["Tweet_Degistirilmis"])):        
                Dataset["Tweet_Degistirilmis"][i]=pattern.sub(' ', Dataset["Tweet_Degistirilmis"][i])
            print ("Tweetler Temizlendi!")
            getTweets.Ceviri(Dataset)
            return Dataset
        except Exception as err:
            print('Error: {}'.format(err))
    
    def KelimeAyristirma(DataSet):
        try:
            data_test = DataSet.copy()
            stop = stopwords.words('english')
            data_test["Tweet_Degistirilmis"] = data_test["Tweet_Degistirilmis"].str.lower().str.split()
            data_test["Tweet_Degistirilmis"] = data_test["Tweet_Degistirilmis"].apply(lambda x: [item for item in x if item not in stop])
        except Exception as err:
            print('Error: {}'.format(err))
    
    def turkceAnaliz(self,Dataset):
        return poyraz.predict(Dataset)
    
    def getSubjectivity(self,Dataset):
        return TextBlob(Dataset).sentiment.subjectivity
    
    def getPolarity(self,Dataset):
        return TextBlob(Dataset).sentiment.polarity 


In [ ]:
tw = getTweets( 
ckey = 'o7qidRODRWm5u07PtAWrCpSqf',
csecret = '1Yn60OYMJl5kRCSM0XQ4Lskxh6hjM4VMz51OdsS6dM9DQuQ4MV',
atoken = '1315792402234441730-Dt7AydgVGYI5VN9RRQfnR8Endh0WAc',
asecret = 'jhfiDfB53Z27EqHfvfKbNdo0w2SWLqyyoAkILqgSoaGMK'
)
aranan=input("Aranacak Kelimeyi Girin: ")
tw.searchTweet(aranan)

Bağlantı Yapıldı!
Kelimeye Uygun Tweetler Sorgulanıyor...
Kelimeye Uygun Tweetler Sorgulandı!
Veriler Kayıt Ediliyor...
Veriler Kayıt Edildi!
Tweetler Temizleniyor...
Tweetler Temizlendi!
Tweetler Çeviriliyor...
Tweetler Çevirildi!


In [ ]:
Veri=pd.read_csv('tw.csv')
Veri.head()

,Tweet_Id,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil
0,1604099286702198784,@Gezegen2022 @valetudinaire_ @MeveRicardo Kıskançlığın kötü bir şey olduğunu öğretmediler mi abicim sana :),Kıskançlığın kötü bir şey olduğunu öğretmediler mi abicim sana,"@Planet2022 @valetudinaire_ @mevericardo did not teach you that jealousy is a bad thing, abicim you :)",2022-12-17 13:00:47+00:00,1336386704693981184,🐞🐐Reyhan💙💛,tr
1,1604097561932988416,destekçimiz bize kötü zamanlar için de yememiz gerektiği konusunda ısrar etmediği #karşıyaka sürece. Bazı madensi https://t.co/iEJzyhBXWM,destekçimiz bize kötü zamanlar için de yememiz gerektiği konusunda ısrar etmediği şıyaka sürece Bazı madensi,Our supporter does not insist that we should eat for bad times. Some mineral https://t.co/iejzyhbxwm,2022-12-17 12:53:56+00:00,1600569922559557633,Hung Kilback,tr
2,1604096524555894786,@tnreskikoyuncu Bizler kötü insanları Ermenilere veya yahudilere benzeterek farkında olmadan ırkçılık ve haksızlık… https://t.co/i8LBiOzmqU,Bizler kötü insanları Ermenilere veya yahudilere benzeterek farkında olmadan ırkçılık ve haksızlık,@tnreskikoyuncu We are unaware of racism and injustice by simulating bad people to Armenians or Jews… https://t.co/i8lbiozmqu,2022-12-17 12:49:48+00:00,3238828983,ahmedt_arif,tr
3,1604096379995000833,Her gün bu siteden daha kötü tweet çıkmaz diyorum her gün daha da kötüsü çıkıyor. Sizi kim bilirkişi yaptı azk https://t.co/scYtJu9ElB,Her gün bu siteden daha kötü tweet çıkmaz diyorum her gün daha da kötüsü çıkıyor Sizi kim bilirkişi yaptı azk,Every day I say worse tweets than this site is worse every day. Who made you an expert azk https://t.co/scytju9elb,2022-12-17 12:49:14+00:00,1566435360309215233,nisa 🔪,tr
4,1604095722156183552,"Evinde , taraftarının önünde hücum futbolundan geri kalmayan Sakaryaspor , formsuz ve kötü futbol oynayan Gençlerbi… https://t.co/Co2tieLnBS",Evinde taraftarının önünde hücum futbolundan geri kalmayan Sakaryaspor formsuz ve kötü futbol oynayan Gençlerbi,"Sakaryaspor, who did not fall behind the offensive football in front of his fans in front of his fans",2022-12-17 12:46:37+00:00,1525855288930320384,Xavi Value,tr


In [ ]:
Veri.tail()

,Tweet_Id,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil
95,1603851817720418323,2 gündür ne oluyo bana ya hiç bu kadar kötü olmamıştım,gündür ne oluyo bana ya hiç bu kadar kötü olmamıştım,What happened to me for 2 days or I've never been so bad,2022-12-16 20:37:26+00:00,1357448010712678402,alaska,tr
96,1603849402631733253,"Hayat bana ne yaşatırsa yaşatsın, karşıma ne kadar kötü insan çıkarsa çıksın, ne kadar canım acırsa acısın, şu üç ş… https://t.co/RO35CreBop",Hayat bana ne yaşatırsa yaşatsın karşıma ne kadar kötü insan çıkarsa çıksın ne kadar canım acırsa acısın şu üç ş,"No matter what life makes me live, no matter how bad people come across me, no matter how much my dear hurts, it is three shit… https://t.co/ro35crebop",2022-12-16 20:27:50+00:00,1492971198321418240,Amigdala,tr
97,1603848418849394704,Kötü biri değilim ama iyi olmaktan yoruldum,Kötü biri değilim ama iyi olmaktan yoruldum,I'm not a bad person but I'm tired of being good,2022-12-16 20:23:55+00:00,1414322005281812484,churchill🇬🇧,tr
98,1603848132147744787,@yasafener_1907 @FBBasketbol @Baskonia Baskan ya basari getirecek takviyeler yapsin takima ya da desinki basketin g… https://t.co/BBoG4Mtl3u,Baskan ya basari getirecek takviyeler yapsin takima ya da desinki basketin g,@yasafener_1907 @fbbasketbol @baskonia to make supplements that will bring success to the team or the baskini baska or the baskona.,2022-12-16 20:22:47+00:00,1410714886565515268,D o b r a 👊,tr
99,1603846555844415500,@fasolarberline Reis sen taraflı olmasan kıyas yaparsın. TC ve Fransa'ya eşit yaklaşırsın. Sen isteğin benimde Türk… https://t.co/sdf9ig2FbA,Reis sen taraflı olmasan kıyas yaparsın TC ve Fransa ya eşit yaklaşırsın Sen isteğin benimde Türk,"@fasolarline Reis you do not compare if you are not biased. You approach the TC and France equally. You are your request, Turkish… https://t.co/sdf9ig2fba",2022-12-16 20:16:31+00:00,1560876723784351746,🇨🇵Santino 🇫🇷#aday-olma-KK,tr


In [ ]:
savedTweets = Veri
savedTweets['Subjectivity'] = savedTweets['Tweet_Ceviri'].apply(tw.getSubjectivity)
savedTweets['Polarity'] = savedTweets['Tweet_Ceviri'].apply(tw.getPolarity)
savedTweets['Duygu Turkce'] = savedTweets['Tweet_Ceviri'].apply(tw.turkceAnaliz)
savedTweets.to_csv('duygu.csv')
savedTweets.head()

,Tweet_Id,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil,Subjectivity,Polarity
0,1604099286702198784,@Gezegen2022 @valetudinaire_ @MeveRicardo Kıskançlığın kötü bir şey olduğunu öğretmediler mi abicim sana :),Kıskançlığın kötü bir şey olduğunu öğretmediler mi abicim sana,"@Planet2022 @valetudinaire_ @mevericardo did not teach you that jealousy is a bad thing, abicim you :)",2022-12-17 13:00:47+00:00,1336386704693981184,🐞🐐Reyhan💙💛,tr,0.833333,-0.10
1,1604097561932988416,destekçimiz bize kötü zamanlar için de yememiz gerektiği konusunda ısrar etmediği #karşıyaka sürece. Bazı madensi https://t.co/iEJzyhBXWM,destekçimiz bize kötü zamanlar için de yememiz gerektiği konusunda ısrar etmediği şıyaka sürece Bazı madensi,Our supporter does not insist that we should eat for bad times. Some mineral https://t.co/iejzyhbxwm,2022-12-17 12:53:56+00:00,1600569922559557633,Hung Kilback,tr,0.666667,-0.70
2,1604096524555894786,@tnreskikoyuncu Bizler kötü insanları Ermenilere veya yahudilere benzeterek farkında olmadan ırkçılık ve haksızlık… https://t.co/i8LBiOzmqU,Bizler kötü insanları Ermenilere veya yahudilere benzeterek farkında olmadan ırkçılık ve haksızlık,@tnreskikoyuncu We are unaware of racism and injustice by simulating bad people to Armenians or Jews… https://t.co/i8lbiozmqu,2022-12-17 12:49:48+00:00,3238828983,ahmedt_arif,tr,0.583333,-0.35
3,1604096379995000833,Her gün bu siteden daha kötü tweet çıkmaz diyorum her gün daha da kötüsü çıkıyor. Sizi kim bilirkişi yaptı azk https://t.co/scYtJu9ElB,Her gün bu siteden daha kötü tweet çıkmaz diyorum her gün daha da kötüsü çıkıyor Sizi kim bilirkişi yaptı azk,Every day I say worse tweets than this site is worse every day. Who made you an expert azk https://t.co/scytju9elb,2022-12-17 12:49:14+00:00,1566435360309215233,nisa 🔪,tr,0.600000,-0.40
4,1604095722156183552,"Evinde , taraftarının önünde hücum futbolundan geri kalmayan Sakaryaspor , formsuz ve kötü futbol oynayan Gençlerbi… https://t.co/Co2tieLnBS",Evinde taraftarının önünde hücum futbolundan geri kalmayan Sakaryaspor formsuz ve kötü futbol oynayan Gençlerbi,"Sakaryaspor, who did not fall behind the offensive football in front of his fans in front of his fans",2022-12-17 12:46:37+00:00,1525855288930320384,Xavi Value,tr,0.700000,-0.40


In [ ]:
savedTweets.tail()

,Tweet_Id,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil,Subjectivity,Polarity
95,1603851817720418323,2 gündür ne oluyo bana ya hiç bu kadar kötü olmamıştım,gündür ne oluyo bana ya hiç bu kadar kötü olmamıştım,What happened to me for 2 days or I've never been so bad,2022-12-16 20:37:26+00:00,1357448010712678402,alaska,tr,0.666667,-0.700000
96,1603849402631733253,"Hayat bana ne yaşatırsa yaşatsın, karşıma ne kadar kötü insan çıkarsa çıksın, ne kadar canım acırsa acısın, şu üç ş… https://t.co/RO35CreBop",Hayat bana ne yaşatırsa yaşatsın karşıma ne kadar kötü insan çıkarsa çıksın ne kadar canım acırsa acısın şu üç ş,"No matter what life makes me live, no matter how bad people come across me, no matter how much my dear hurts, it is three shit… https://t.co/ro35crebop",2022-12-16 20:27:50+00:00,1492971198321418240,Amigdala,tr,0.455556,-0.121212
97,1603848418849394704,Kötü biri değilim ama iyi olmaktan yoruldum,Kötü biri değilim ama iyi olmaktan yoruldum,I'm not a bad person but I'm tired of being good,2022-12-16 20:23:55+00:00,1414322005281812484,churchill🇬🇧,tr,0.655556,0.216667
98,1603848132147744787,@yasafener_1907 @FBBasketbol @Baskonia Baskan ya basari getirecek takviyeler yapsin takima ya da desinki basketin g… https://t.co/BBoG4Mtl3u,Baskan ya basari getirecek takviyeler yapsin takima ya da desinki basketin g,@yasafener_1907 @fbbasketbol @baskonia to make supplements that will bring success to the team or the baskini baska or the baskona.,2022-12-16 20:22:47+00:00,1410714886565515268,D o b r a 👊,tr,0.000000,0.300000
99,1603846555844415500,@fasolarberline Reis sen taraflı olmasan kıyas yaparsın. TC ve Fransa'ya eşit yaklaşırsın. Sen isteğin benimde Türk… https://t.co/sdf9ig2FbA,Reis sen taraflı olmasan kıyas yaparsın TC ve Fransa ya eşit yaklaşırsın Sen isteğin benimde Türk,"@fasolarline Reis you do not compare if you are not biased. You approach the TC and France equally. You are your request, Turkish… https://t.co/sdf9ig2fba",2022-12-16 20:16:31+00:00,1560876723784351746,🇨🇵Santino 🇫🇷#aday-olma-KK,tr,0.250000,0.000000
